In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
import tensorflow as tf
tf.keras.optimizers.Adamax(learning_rate=1e-2)

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

from tensorflow.keras.layers import Dense, Input, ZeroPadding1D
from tensorflow.keras.layers import Flatten, BatchNormalization
from tensorflow.keras.layers import Reshape
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.losses import mse, binary_crossentropy
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Layer
#from keras import objectives
from tensorflow.python.client import device_lib
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

import numpy as np
import matplotlib.pyplot as plt
import argparse
import os
import math
import time
import h5py
import POD_initial as POD_initial

In [2]:
scalar = 'T'

mins = [0.8,0.8,0.8,0.05,0.8,1,1,1,1,1,1,1,20,150,0.1,0.1,0.001,0.001,0.1,0.1,0.1,0.1,1,1,7,0.5,0.5,2,5,5,53,30,65,10,10,10,10,1280,0.06,0]
maxs = [0.95,0.95,0.95,0.2,0.95,10,10,10,10,10,10,15,115,1000,1,1,0.1,0.1,1,1,1,1,5,5,12,1,1,7,10,10,58,36,70,15,15,15,15,1420,0.13,90]

snap_path = 'Training_output/14OD{0}.txt'
test_path = 'Test_output/14OV{0}.txt'

n_modes = 7

pod_p = open('POD_initial/{0}/{1}'.format(1,scalar), 'r')
pod_p_lines = pod_p.readlines()
int_length = len(pod_p_lines)
orth_basis = np.zeros((int_length,n_modes))

# read POD mode
for i in range(n_modes):
    pod_p = open('POD_initial/{0}/{1}'.format(i+1,scalar), 'r')
    pod_p_lines = pod_p.readlines()
    pod_size = 0
    for j in range(int_length):
        pod_p_line = float(pod_p_lines[j])
        pod_size = pod_size + pod_p_line*pod_p_line

    for j in range(int_length):
        pod_p_line = float(pod_p_lines[j])
        orth_basis[j,i] =pod_p_line/math.sqrt(pod_size)

# read X_unlabeled
parameters_unlabeled = open('training_input_unlabeled.dat','r')
parameters_lines_unlabeled = parameters_unlabeled.readlines()
n_parameters_unlabeled = len(parameters_lines_unlabeled[0].split())
n_snaps_unlabeled = len(parameters_lines_unlabeled)
X_u = np.zeros((n_snaps_unlabeled,n_parameters_unlabeled))
for k in range(n_snaps_unlabeled):
    parameters_line_unlabeled = parameters_lines_unlabeled[k].split()
    for kp in range(n_parameters_unlabeled):
        X_u[k][kp]=(float(parameters_line_unlabeled[kp])-mins[kp])/(maxs[kp]-mins[kp])
        

# read parameters
parameters = open('training_input.dat','r')
parameters_lines = parameters.readlines()
n_parameters = len(parameters_lines[0].split())
n_snaps = len(parameters_lines)
X_p = np.zeros((n_snaps,n_parameters))
for k in range(n_snaps):
    parameters_line = parameters_lines[k].split()
    for kp in range(n_parameters):
        X_p[k][kp]=(float(parameters_line[kp])-mins[kp])/(maxs[kp]-mins[kp])

# make a parameter matrix  
X_train_full = X_p
X = X_train_full
X_unlabeled = X_u
print(X.shape)
print(X_unlabeled.shape)

# make a parameter matrix for prediction
v_parameters = open('test_input.dat','r')

v_parameters_lines = v_parameters.readlines()
n_valids = len(v_parameters_lines)

X_v = np.zeros((n_valids, n_parameters))
for k in range(n_valids):
    v_parameters_line = v_parameters_lines[k].split()
    for kp in range(n_parameters):
        X_v[k][kp] = (float(v_parameters_line[kp]) - mins[kp]) / (maxs[kp] - mins[kp])
X_test = X_v
print(X_test.shape)
        
# read snapshots
snaps = np.zeros((int_length,n_snaps))
for k in range(n_snaps):
    snap_p = open(snap_path.format(k+1,scalar),'r')
    snap_p_lines = snap_p.readlines()

    for j in range(int_length):
        snap_p_line = float(snap_p_lines[j])
        snaps[j,k] = snap_p_line        
        
# calculate coefficient alpha
alpha = np.zeros((n_modes,n_snaps))
for i in range(n_modes):
    for k in range(n_snaps):
        alpha[i][k] = np.inner(orth_basis[:,i],snaps[:,k])

# Transposition for training DNN (Regression model)
alpha = np.transpose(alpha)
alpha_norm = np.zeros((alpha.shape))

# normalize POD coefficient (training coeffs)
for k in range(n_snaps):
    for kp in range(n_modes):
        alpha_norm[k][kp] = ( alpha[k][kp] - np.min(alpha[:,kp])) / (np.max(alpha[:,kp]) - np.min(alpha[:,kp]))
        
coeff_dim = n_modes 
y_train_full = alpha_norm[:100]
y = y_train_full
print(y.shape)

X_train, X_val, y_train, y_val = train_test_split(X_train_full, y_train_full, test_size=0.2, random_state=42)
print(X_train.shape)
print(y_train.shape)
print(X_val.shape)
print(y_val.shape)

(20, 40)
(4000, 40)
(50, 40)
(20, 7)
(16, 40)
(16, 7)
(4, 40)
(4, 7)


In [3]:
# Hyperparameter grid
hidden_layers_options = [2, 3, 4]  # Number of hidden layers
neurons_options = [40, 80, 160, 320, 640, 1280]
batch_size_options = [5, 10, 20]  # Batch sizes
learning_rate_options = [0.01, 0.001, 0.0001]  # Learning rates

In [4]:
def build_model(hidden_layers, neurons, learning_rate):
    model = Sequential()
    model.add(Dense(neurons, input_dim=X_train.shape[1], activation='relu'))
    model.add(BatchNormalization())
    for _ in range(hidden_layers - 1):
        model.add(Dense(neurons, activation='relu'))
        model.add(BatchNormalization())
    model.add(Dense(coeff_dim))  # Output layer for regression
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='mse')
    return model

In [5]:
"""
# Grid search
best_score = np.inf
best_params = {}

for hidden_layers in hidden_layers_options:
    for neurons in neurons_options:
        for batch_size in batch_size_options:
            for learning_rate in learning_rate_options:
                print(f"Testing model with {hidden_layers} hidden layers, {neurons} neurons, batch size {batch_size}, and learning rate {learning_rate}")
                model = build_model(hidden_layers, neurons, learning_rate)
                model.fit(X_train, y_train, batch_size=batch_size, epochs=50, verbose=0)
                score = model.evaluate(X_val, y_val, verbose=0)
                print(f"Test MSE: {score}")

                if score < best_score:
                    best_score = score
                    best_params = {'hidden_layers': hidden_layers, 'neurons': neurons, 'batch_size': batch_size, 'learning_rate': learning_rate}

print(f"Best score (MSE): {best_score}")
print(f"Best parameters: {best_params}")
"""

'\n# Grid search\nbest_score = np.inf\nbest_params = {}\n\nfor hidden_layers in hidden_layers_options:\n    for neurons in neurons_options:\n        for batch_size in batch_size_options:\n            for learning_rate in learning_rate_options:\n                print(f"Testing model with {hidden_layers} hidden layers, {neurons} neurons, batch size {batch_size}, and learning rate {learning_rate}")\n                model = build_model(hidden_layers, neurons, learning_rate)\n                model.fit(X_train, y_train, batch_size=batch_size, epochs=50, verbose=0)\n                score = model.evaluate(X_val, y_val, verbose=0)\n                print(f"Test MSE: {score}")\n\n                if score < best_score:\n                    best_score = score\n                    best_params = {\'hidden_layers\': hidden_layers, \'neurons\': neurons, \'batch_size\': batch_size, \'learning_rate\': learning_rate}\n\nprint(f"Best score (MSE): {best_score}")\nprint(f"Best parameters: {best_params}")\n'

In [6]:
# Build and train the model with the optimal hyperparameters
best_params = {'hidden_layers': 2, 'neurons': 80, 'batch_size': 5, 'learning_rate': 0.01}

# Step 4: k-Fold Cross-Validation
kf = KFold(n_splits=5, shuffle=True, random_state=42)
cv_losses = []

for train_index, val_index in kf.split(X):
    X_train_cv, X_val_cv = X[train_index], X[val_index]
    y_train_cv, y_val_cv = y[train_index], y[val_index]
    model = build_model(best_params['hidden_layers'], best_params['neurons'], best_params['learning_rate'])
    model.fit(X_train_cv, y_train_cv, epochs=50, batch_size=5, verbose=1)
    val_loss = model.evaluate(X_val_cv, y_val_cv, verbose=0)
    cv_losses.append(val_loss)

average_cv_loss = np.mean(cv_losses)
print(f"Average CV Loss: {average_cv_loss}")

Epoch 1/50


C:\Anaconda3\envs\test02\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 2.0815
Epoch 2/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.2355 
Epoch 3/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.8546 
Epoch 4/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.6033 
Epoch 5/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.4260 
Epoch 6/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.4389 
Epoch 7/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.2936 
Epoch 8/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.3523 
Epoch 9/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.3226 
Epoch 10/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.2292 
Epoch 11/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.1702 
Epoch 12/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.1941 
Epoch 13/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.2623 
Epoch 14/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.1918 
Epoch 15/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.1513 
Epoch 16/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/st

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0634 
Epoch 35/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0700 
Epoch 36/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0687 
Epoch 37/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0699 
Epoch 38/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0762 
Epoch 39/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0481 
Epoch 40/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0534 
Epoch 41/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0556 
Epoch 42/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0405 
Epoch 43/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0462 
Epoch 44/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0494 
Epoch 45/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0448 
Epoch 46/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0485 
Epoch 47/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0513 
Epoch 48/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0513 
Epoch 49/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.1189 
Epoch 18/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.1778 
Epoch 19/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.1059 
Epoch 20/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.1083 
Epoch 21/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0976 
Epoch 22/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0941 
Epoch 23/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0924 
Epoch 24/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0997 
Epoch 25/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.1021 
Epoch 26/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0880 
Epoch 27/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.1019 
Epoch 28/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.1149 
Epoch 29/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0945 
Epoch 30/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0656 
Epoch 31/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0837 
Epoch 32/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0442 
Epoch 50/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0474 
Average CV Loss: 0.11684487164020538


In [7]:
# Step 5: Train the Final Model
X_final_train = X
y_final_train = y
final_models = [build_model(best_params['hidden_layers'], best_params['neurons'], best_params['learning_rate']) for _ in range(50)]

# Training each model
for final_model in final_models:
    history = final_model.fit(
        X_final_train, y_final_train,
        batch_size=best_params['batch_size'],
        epochs=250, # Set to a large number, early stopping will determine the actual number of epochs
        verbose=1
    )
    print("network was trained")

Epoch 1/250
4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 1.9037
Epoch 2/250
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.6043 
Epoch 3/250
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.8294 
Epoch 4/250
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.6254 
Epoch 5/250
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5165 
Epoch 6/250
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.2874 
Epoch 7/250
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.2391 
Epoch 8/250
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.2992 
Epoch 9/250
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.2545 
Epoch 10/250
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.2049 
Epoch 11/250
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.1794 
Epoch 12/250
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.1480 
Epoch 13/250
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.1225 
Epoch 14/250
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.1473 
Epoch 15/250
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.1267 
Epoch 16/250
4/4 ━━━

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0214 
Epoch 84/250
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0188 
Epoch 85/250
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0220 
Epoch 86/250
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0254 
Epoch 87/250
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0266 
Epoch 88/250
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0283 
Epoch 89/250
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0173 
Epoch 90/250
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0218 
Epoch 91/250
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0345 
Epoch 92/250
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0275 
Epoch 93/250
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0197 
Epoch 94/250
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0297 
Epoch 95/250
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0244 
Epoch 96/250
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0246 
Epoch 97/250
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0195 
Epoch 98/250
4/4 ━━━━━━

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0100 
Epoch 165/250
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0161 
Epoch 166/250
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0178 
Epoch 167/250
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0151 
Epoch 168/250
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0166 
Epoch 169/250
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0201 
Epoch 170/250
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0133 
Epoch 171/250
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0180 
Epoch 172/250
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0136 
Epoch 173/250
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0164 
Epoch 174/250
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0223 
Epoch 175/250
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0182 
Epoch 176/250
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0156 
Epoch 177/250
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0103 
Epoch 178/250
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0217 
Epoch 179

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0145 
Epoch 246/250
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0147 
Epoch 247/250
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0102 
Epoch 248/250
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0133 
Epoch 249/250
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0156 
Epoch 250/250
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0162 
network was trained
Epoch 1/250
4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 1.7071
Epoch 2/250
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1.2398 
Epoch 3/250
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.8989 
Epoch 4/250
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5102 
Epoch 5/250
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.4382 
Epoch 6/250
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.4247 
Epoch 7/250
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.2416 
Epoch 8/250
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.3153 
Epoch 9/250
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.3297 
Epoch 10

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0225 
Epoch 77/250
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0247 
Epoch 78/250
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0198 
Epoch 79/250
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0309 
Epoch 80/250
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0167 
Epoch 81/250
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0285 
Epoch 82/250
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0262 
Epoch 83/250
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0306 
Epoch 84/250
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0238 
Epoch 85/250
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0252 
Epoch 86/250
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0368 
Epoch 87/250
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0268 
Epoch 88/250
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0230 
Epoch 89/250
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0302 
Epoch 90/250
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0207 
Epoch 91/250
4/4 ━━━━━━

...

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0210 
Epoch 169/250
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0178 
Epoch 170/250
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0102 
Epoch 171/250
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0196 
Epoch 172/250
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0121 
Epoch 173/250
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0197 
Epoch 174/250
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0207 
Epoch 175/250
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0151 
Epoch 176/250
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0202 
Epoch 177/250
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0233 
Epoch 178/250
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0105 
Epoch 179/250
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0175 
Epoch 180/250
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0165 
Epoch 181/250
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0169 
Epoch 182/250
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0231 
Epoch 183

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0134 
Epoch 250/250
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0161 
network was trained
Epoch 1/250
4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 2.1742
Epoch 2/250
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.1143 
Epoch 3/250
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.6778 
Epoch 4/250
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5664 
Epoch 5/250
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.3794 
Epoch 6/250
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.3202 
Epoch 7/250
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.2991 
Epoch 8/250
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.2162 
Epoch 9/250
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.3104 
Epoch 10/250
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.2181 
Epoch 11/250
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.2347 
Epoch 12/250
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.1300 
Epoch 13/250
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.1530 
Epoch 14/250

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0209 
Epoch 81/250
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0285 
Epoch 82/250
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0267 
Epoch 83/250
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0273 
Epoch 84/250
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0318 
Epoch 85/250
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0209 
Epoch 86/250
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0263 
Epoch 87/250
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0269 
Epoch 88/250
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0212 
Epoch 89/250
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0226 
Epoch 90/250
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0260 
Epoch 91/250
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0273 
Epoch 92/250
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0195 
Epoch 93/250
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0232 
Epoch 94/250
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0176 
Epoch 95/250
4/4 ━━━━━━

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0176 
Epoch 162/250
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0182 
Epoch 163/250
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0175 
Epoch 164/250
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0225 
Epoch 165/250
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0173 
Epoch 166/250
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0159 
Epoch 167/250
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0216 
Epoch 168/250
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0219 
Epoch 169/250
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0205 
Epoch 170/250
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0183 
Epoch 171/250
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0155 
Epoch 172/250
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0199 
Epoch 173/250
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0130 
Epoch 174/250
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0289 
Epoch 175/250
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0096 
Epoch 176

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0218 
Epoch 243/250
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0169 
Epoch 244/250
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0132 
Epoch 245/250
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0181 
Epoch 246/250
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0193 
Epoch 247/250
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0155 
Epoch 248/250
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0160 
Epoch 249/250
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0181 
Epoch 250/250
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0098 
network was trained


In [8]:
norm_p_alpha = np.mean([final_model.predict(X_test) for final_model in final_models], axis=0)

# Denormalize POD coefficient
p_alpha = np.zeros((norm_p_alpha.shape))
for kp in range(len(np.transpose(alpha))):
    p_alpha[:,kp] = norm_p_alpha[:,kp] * (np.max(alpha[:,kp]) - np.min(alpha[:,kp])) + np.min(alpha[:,kp])

if n_valids == 1:
    recons_p_field = np.matmul(basis,np.transpose(p_alpha))
    recons_p_field = recons_p_field.flatten()

else:
    recons_p_field = np.matmul(orth_basis,np.transpose(p_alpha))

sol = np.zeros((int_length, n_valids))

for kp in range(n_valids):
    for i in range(int_length):
        sol[i, kp] = recons_p_field[i, kp]

# build predicted fields
if not os.path.isdir('predict_initial'):
    os.mkdir('predict_initial')
for k in range(n_valids):
    print('Predict {0} for parameter{1}'.format(scalar, k + 1))
    if not os.path.isdir('predict_initial/{0}'.format(k + 1)):
        os.mkdir('predict_initial/{0}'.format(k + 1))
    reconst_p = open('predict_initial/{0}/{1}'.format(k + 1, scalar), 'w')

    if n_valids == 1:
        sol.flatten()
        for i in range(int_length):
            rec_p = sol[i]
            reconst_p.write(str(rec_p))
            reconst_p.write('\n')

    else:
        for i in range(int_length):
            rec_p = sol[i, k]
            reconst_p.write(str(rec_p))
            reconst_p.write('\n')
reconst_p.close()


2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step
1/2 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/stepWARNING:tensorflow:6 out of the last 6 calls to <function TensorFlowTrainer.make_predict_function.<locals>.one_step_on_data_distributed at 0x00000186DC6DDA80> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings could be due to (1) creating @tf.function repeatedly in a loop, (2) passing tensors with different shapes, (3) passing Python objects instead of tensors. For (1), please define your @tf.function outside of the loop. For (2), @tf.function has reduce_retracing=True option that can avoid unnecessary retracing. For (3), please refer to https://www.tensorflow.org/guide/function#controlling_retracing and https://www.tensorflow.org/api_docs/python/tf/function for  more details.
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step
2/2 ━━━━━

In [10]:
n_validation = 50

cfd_path = 'predict_initial/{0}/{1}'
val_path = 'Test_output/14OV{0}.txt'

int_length = 1666 #numbers of grid
scalar = 'T'

predict = np.zeros((int_length,n_validation))
diff = np.zeros((int_length,n_validation))
absol = np.zeros((int_length,n_validation))
errsqr = np.zeros((int_length,n_validation))
valsqr = np.zeros((int_length,n_validation))
valsqrsum = np.zeros((n_validation))
errsqrsum = np.zeros((n_validation))
val_mean = np.zeros((n_validation))
val_mean1 = np.ones((int_length, n_validation))
sqrsum = np.zeros((int_length,n_validation))
totsqrsum = np.zeros((n_validation))

snap_stack_val = np.zeros((int_length, n_validation))

for val_num in range(n_validation):
    predict_file = open(cfd_path.format(val_num+1,scalar),'r')
#   predict_file = open('predict/{0}/{1}'.format(val_num+1,scalar),'r')
    predict_file_lines = predict_file.readlines()
    snap_file = open(val_path.format(val_num+1,scalar),'r')
    snap_file_lines = snap_file.readlines()
    for i in range(int_length):
        predict_file_line = float(predict_file_lines[i])
        predict[i,val_num] = predict_file_line
        snap_file_line = float(snap_file_lines[i])
        snap_stack_val[i,val_num] = snap_file_line
        
#        diff[i,val_num] = snap_stack[i,val_num] - predict[i,val_num]
        diff[i,val_num] = snap_stack_val[i,val_num] - predict[i,val_num]
        absol[i,val_num] = abs(diff[i,val_num])
        errsqr[i,val_num] = diff[i,val_num] * diff[i,val_num]
#        valsqr[i,val_num] = snap_stack[i,val_num] * snap_stack[i,val_num]
        valsqr[i,val_num] = snap_stack_val[i,val_num] * snap_stack_val[i,val_num]
    errsqrsum[val_num] = np.sum(errsqr[:,val_num])
    valsqrsum[val_num] = np.sum(valsqr[:,val_num])
#    val_mean[val_num] = np.mean(snap_stack[:,val_num])
    val_mean[val_num] = np.mean(snap_stack_val[:,val_num])
    
    val_mean1[:,val_num] = val_mean1[:,val_num] * val_mean[val_num]
    for i in range(int_length):
#        sqrsum[i, val_num] = (snap_stack[i,val_num] - val_mean1[i,val_num]) * (snap_stack[i,val_num] - val_mean1[i,val_num])
        sqrsum[i, val_num] = (snap_stack_val[i,val_num] - val_mean1[i,val_num]) * (snap_stack_val[i,val_num] - val_mean1[i,val_num])

    totsqrsum[val_num] = np.sum(sqrsum[:,val_num])

rmse = np.zeros((1,n_validation))
nrmse = np.zeros((1,n_validation))
mae = np.zeros((1,n_validation))
mape = np.zeros((1,n_validation))
r2 = np.zeros((1,n_validation))
l2rel = np.zeros((1,n_validation))

for i in range(n_validation):
#    rmse[0,i] = round(math.sqrt(mean_squared_error(snap_stack[:,i],predict[:,i])),8)
    rmse[0,i] = round(math.sqrt(mean_squared_error(snap_stack_val[:,i],predict[:,i])),8)
#    nrmse[0,i] = round(rmse[0,i]*100/(np.mean(snap_stack[:,i])),8)
    nrmse[0,i] = round(rmse[0,i]*100/(np.mean(snap_stack_val[:,i])),8)
    #nrmse[0,i] = round(rmse[0,i]*100/(np.max(val[:,i])-np.min(val[:,i])),8)
#    mape[0,i] = round(100*np.sum(absol[:,i]/snap_stack[:,i])/n_mask,8)
    mape[0,i] = round(100*np.sum(absol[:,i]/snap_stack_val[:,i])/int_length,8)
    mae[0,i] = np.mean(absol[:,i])
    r2[0,i] =  1 - ( errsqrsum[i] / totsqrsum[i])
    l2rel[0,i] = math.sqrt( errsqrsum[i] / valsqrsum[i] )*100	

if not os.path.isdir('error_initial'):
    os.mkdir('error_initial')

err_file = open('error_initial/error_initial_{0}'.format(scalar),'w')
err_file.write('RMSE')
err_file.write('\t\t\t\t')
err_file.write('MAE')
err_file.write('\t\t\t\t')
err_file.write('MAX Error')
err_file.write('\t\t\t\t')
err_file.write('R2')
err_file.write('\t\t\t\t')
err_file.write('L2 relative norm')
err_file.write('\n')

for val_num in range(n_validation):
    
    print('Error check for validation {0}\n'.format(val_num+1))
    print('RMSE : {0}'.format(abs(rmse[0,val_num])))
    print('MAE : {0}'.format(abs(mae[0,val_num])))
    print('Max Error : {0}'.format(round(np.max(absol[:,val_num]),8)))
    print('R2 : {0}'.format(r2[0,val_num]))
    print('L2 relative norm : {0}\n'.format(l2rel[0,val_num]))

    err_file.write(str(rmse[0,val_num]))
    err_file.write('\t\t\t\t')
    err_file.write(str(mae[0,val_num]))
    err_file.write('\t\t\t\t')
    err_file.write(str(np.max(absol[:,val_num])))
    err_file.write('\t\t\t\t')
    err_file.write(str(r2[0,val_num]))
    err_file.write('\t\t\t\t')
    err_file.write(str(l2rel[0,val_num]))
    err_file.write('\n')

print('Average RMSE  : {0}%'.format(round(np.mean(abs(nrmse[0,:])),8)))
print('Average R2  : {0}'.format(round(np.mean(r2[0,:]),5)))
print('Average L2 relative norm  : {0}'.format(round(np.mean(l2rel[0,:]),5)))
err_file.close()

Error check for validation 1

RMSE : 3.14722655
MAE : 3.0526410161276676
Max Error : 4.27234773
R2 : 0.7358298459383286
L2 relative norm : 1.062026454631509

Error check for validation 2

RMSE : 2.05382533
MAE : 1.9097649937110985
Max Error : 4.02461974
R2 : 0.8741859406150868
L2 relative norm : 0.6831507616698892

Error check for validation 3

RMSE : 1.22642678
MAE : 0.881877407814745
Max Error : 3.65401761
R2 : 0.9435063338237083
L2 relative norm : 0.41647052612974655

Error check for validation 4

RMSE : 5.31220142
MAE : 4.997140731008323
Max Error : 7.76596547
R2 : 0.16634996938467295
L2 relative norm : 1.7443669065094671

Error check for validation 5

RMSE : 2.91797977
MAE : 2.6409834660343847
Max Error : 4.51144674
R2 : 0.8372117983182166
L2 relative norm : 0.9785667394647224

Error check for validation 6

RMSE : 4.09072984
MAE : 3.495108108509309
Max Error : 6.87269669
R2 : 0.09609006536364395
L2 relative norm : 1.369231284966325

Error check for validation 7

RMSE : 4.29659615


In [11]:
num_iterations = 4  # Number of active learning iterations
num_samples_per_query = 20  # Number of samples to query in each iteration

# Placeholder for predictions and uncertainty calculation
predictions_unlabeled = np.array([final_model.predict(X_unlabeled) for final_model in final_models])
print(predictions_unlabeled.shape)
mean_preds = np.mean(predictions_unlabeled, axis=0)
std_devs = np.std(predictions_unlabeled, axis=0)
uncertainty = np.mean(std_devs, axis=1)
print(mean_preds.shape)
print(std_devs.shape)
print(uncertainty.shape)

# Select samples for labeling based on highest uncertainty
query_indices = np.argsort(-uncertainty)[:num_samples_per_query]
print(query_indices)
X_queried = X_unlabeled[query_indices]
print(X_queried.shape)

X_queried_scale = np.zeros((X_queried.shape))
for kp in range(n_parameters):
    X_queried_scale[:,kp] = X_queried[:,kp] * (maxs[kp] - mins[kp]) + mins[kp]

np.savetxt("Uncertainty_1.txt", uncertainty)
np.savetxt("query_indices_1.txt", query_indices)
np.savetxt("X_queried_1.txt", X_queried_scale)


125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 633us/step
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 823us/step
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 682us/step
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 649us/step
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 631us/step
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 635us/step
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 622us/step
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 650us/step
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 638us/step
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 719us/step
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 713us/step
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 656us/step
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 805us/step
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 763us/step
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 705us/step
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 757us/step
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 705us/step
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 737us/step
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 674us/step
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 623us/step
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 707us/step
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 711us/step
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 697us/step
125/125 ━━━